In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import os
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.layers import AveragePooling2D

In [3]:
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test)= cifar10.load_data()

In [5]:
x_train = np.concatenate((x_train,x_test), axis=0)
y_train = np.concatenate((y_train,y_test), axis=0)

In [6]:
seed = 3
np.random.seed(seed)
a = np.random.randint(60000,size=12000)
x_train = x_train[a]
y_train = y_train[a]
x_train.shape
y_train.shape

(12000, 1)

In [7]:
num_classes = 10

In [8]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [9]:
model = Sequential()

In [10]:
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [11]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [12]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [13]:
### using SGD as optimizer 
opt = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [14]:
# Let's train the model using SGD
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [16]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [17]:
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

In [18]:
    history = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=32),
                        steps_per_epoch=x_train.shape[0] // 32,
                        epochs=5,
                        validation_data=(x_test, y_test),
                        workers=4)

Epoch 1/5
375/375 [==============================] - 21s - loss: 2.1314 - acc: 0.2037 - val_loss: 1.9262 - val_acc: 0.2779
Epoch 2/5
375/375 [==============================] - 20s - loss: 1.9235 - acc: 0.2993 - val_loss: 1.7331 - val_acc: 0.3745
Epoch 3/5
375/375 [==============================] - 20s - loss: 1.7841 - acc: 0.3472 - val_loss: 1.5999 - val_acc: 0.4269
Epoch 4/5
375/375 [==============================] - 20s - loss: 1.6700 - acc: 0.3800 - val_loss: 1.4962 - val_acc: 0.4493
Epoch 5/5
375/375 [==============================] - 20s - loss: 1.5723 - acc: 0.4216 - val_loss: 1.4353 - val_acc: 0.4797


In [19]:
score = model.evaluate(x_test, y_test, verbose=0)

In [20]:
score

[1.4353483554840087, 0.47970000000000002]

In [21]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /gpfs/global_fs01/sym_shared/YPProdSpark/user/s45f-65f67acefe15fa-238764f4c881/notebook/work/saved_models/keras_cifar10_trained_model.h5 


In [22]:

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'

In [23]:
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [25]:

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=32,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // 32,
                                      workers=4)

In [26]:
i=0
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if actual_label == predicted_label: print(i)
    if predict_index == num_predictions:
        break
        i

Actual Label = cat vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = horse
Actual Label = airplane vs. Predicted Label = cat
Actual Label = frog vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = ship
Actual Label = automobile vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = cat
Actual Label = cat vs. Predicted Label = truck
Actual Label = automobile vs. Predicted Label = frog
Actual Label = airplane vs. Predicted Label = dog
Actual Label = truck vs. Predicted Label = deer
Actual Label = dog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Label = ship
Actual Label = truck vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = deer
Actual Label = horse vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Lab